In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#####  IMAGE ONE

In [ ]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import os
import torch
import torchvision
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import albumentations

In [ ]:
import torch
import torch.nn as nn
from skimage import io, transform

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,image_path,features,targets,augmentations=None):
        self.image_path = image_path
        self.features = features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_path)
    
    def __getitem__(self,item):
        image = io.imread(self.image_path[item])
        features = self.features[item,:]
        targets = self.targets[item]
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
import torch
import torch.nn as nn

model_name = 'swin_large_patch4_window12_384'

out_dim    = 1

class PawpularModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False, in_chans=3)
        self.model.head = nn.Linear(self.model.head.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(140, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, image, features, targets=None):
        x = self.model(image)
        x = self.dropout(x)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
        y = x
        x = self.dense2(x)
        return x, y

In [ ]:
import os
from sklearn import metrics
import albumentations
device = 'cuda'
epochs = 12
data_path = '../input/petfinder-pawpularity-score'

valid_aug = albumentations.Compose(
    [
        albumentations.Resize(384, 384, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [ ]:
feats = [
    'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
    'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
]

In [ ]:
from itertools import chain
device = 'cuda'
model_f = PawpularModel(model_name="swin_large_patch4_window12_384")
model_f.to(device)
df_train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
df_test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_outputs_sub = []
train_outputs_sub = []
for _fold in range(10):
    
    model_f.load_state_dict(torch.load('../input/tez-pawpular-swin-ference-model-saves/abhishek-models'+str(_fold)+'.pth'))
    data_path = '../input/petfinder-pawpularity-score'
    device = 'cuda'
    
    train_images = df_train.Id.values.tolist()
    train_images = [os.path.join(data_path,'train',i + '.jpg') for i in train_images]
    test_images = df_test.Id.values.tolist()
    test_images = [os.path.join(data_path,'test',i + '.jpg') for i in test_images]
    
    train_dataset =  CustomDataset(image_path = train_images,features=df_train[feats].values,targets = np.ones(len(train_images)),augmentations=valid_aug)
    train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=1,shuffle=False)
    
    test_dataset =  CustomDataset(image_path = test_images,features=df_test[feats].values,targets = np.ones(len(test_images)),augmentations=valid_aug)
    test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=1,shuffle=False) 
    

    train_outputs = []
    test_outputs = []
    with torch.no_grad():
        for data in train_loader:
            inputs = data['image']
            features=data['features']
            inputs = inputs.to(device, dtype=torch.float)
            features=features.to(device, dtype=torch.float)
            _ ,output = model_f(inputs,features)
            output = (torch.sigmoid(output).detach().cpu().numpy()*100).tolist()
            train_outputs.extend(output)
    
        for data in test_loader:
            inputs = data['image']
            features=data['features']
            inputs = inputs.to(device, dtype=torch.float)
            features=features.to(device, dtype=torch.float)
            _ ,output = model_f(inputs,features)
            output = (torch.sigmoid(output).detach().cpu().numpy()*100).tolist()
            test_outputs.extend(output)
    train_outputs = list(chain.from_iterable(train_outputs)) 
    train_outputs_sub.append(train_outputs)
    
    test_outputs = list(chain.from_iterable(test_outputs)) 
    test_outputs_sub.append(test_outputs)
    
train_outputs_sub = np.mean(np.column_stack(train_outputs_sub), axis=1)
test_outputs_sub = np.mean(np.column_stack(test_outputs_sub), axis=1)
np.savez("train_features", train_outputs_sub)
np.savez("test_features", test_outputs_sub)